<a href="https://colab.research.google.com/github/njambi-r/nairobi-rail-brt-transit-map/blob/main/formatted_v1_TubeMap_NCR_stations_to_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code converts my data to json

In [ ]:
import csv
import json
import math
from collections import defaultdict

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
ALLOWED_DIRECTIONS = [
    (1, 0),    # →
    (0, 1),    # ↓
    (-1, 0),   # ←
    (0, -1),   # ↑
    (1, 1),    # ↘
    (-1, -1),  # ↖
    (1, -1),   # ↗
    (-1, 1),   # ↙
]

In [ ]:
"""
def vector_direction(dx, dy):
    norm = (dx, dy)
    return norm in ALLOWED_DIRECTIONS
"""


#updated version to allow grom movement in any units on the right direction

def normalize_direction(dx, dy):
    if dx == 0 and dy == 0:
        return None
    gcd = math.gcd(dx, dy)
    return (dx // gcd, dy // gcd)

def vector_direction(dx, dy):
    dir = normalize_direction(dx, dy)
    return dir in ALLOWED_DIRECTIONS


In [ ]:
def read_csv(path):
    with open(path, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        return list(reader)

In [ ]:
def scale_coords(data, grid_size=300):
    min_x = min(float(row['x_coord']) for row in data)
    min_y = min(float(row['y_coord']) for row in data)

    grid_positions = {}
    for row in data:
        name = row['Name']
        # Snap to nearest grid_size
        x = round((float(row['x_coord']) - min_x) / grid_size)
        y = round((float(row['y_coord']) - min_y) / grid_size)
        grid_positions[(name, row['NCR_Line'])] = (x, y)

    return grid_positions

In [ ]:
from collections import defaultdict

ALLOWED_DIRECTIONS = {
    (-1, 0), (1, 0),   # horizontal
    (0, -1), (0, 1),   # vertical
    (-1, -1), (1, 1),  # diagonal /
    (-1, 1), (1, -1)   # diagonal \
}

def normalize_direction(dx, dy):
    if dx == 0 and dy == 0:
        return (0, 0)
    return (
        dx // abs(dx) if dx != 0 else 0,
        dy // abs(dy) if dy != 0 else 0
    )

def build_json(data, grid_positions):
    stations_meta = {}
    lines = defaultdict(list)

    # Build initial line data and station metadata
    for row in data:
        key = (row['Name'], row['NCR_Line'])
        x, y = grid_positions[key]
        x, y = int(x), int(y)

        lines[row['NCR_Line']].append({
            'name': row['Name'],
            'coords': [x, y],
            'labelPos': row.get('labelPos', 'S')
        })

        stations_meta.setdefault(row['Name'], {
            'label': row['Name'],
            'position': {
                'lat': float(row['y_coord']),
                'lon': float(row['x_coord'])
            }
        })

    # Smooth lines without inserting corner points unnecessarily
    for line_name, nodes in lines.items():
        smoothed = [nodes[0]]
        for i in range(1, len(nodes) - 1):
            prev = smoothed[-1]
            curr = nodes[i]
            next_node = nodes[i + 1]

            x0, y0 = prev['coords']
            x1, y1 = curr['coords']
            x2, y2 = next_node['coords']

            dir1 = normalize_direction(x1 - x0, y1 - y0)
            dir2 = normalize_direction(x2 - x1, y2 - y1)

            if dir1 not in ALLOWED_DIRECTIONS or dir2 not in ALLOWED_DIRECTIONS:
                print(f"⚠️ Skipping invalid direction: {prev['coords']} → {curr['coords']} → {next_node['coords']}")
                smoothed.append(curr)
                continue

            # No corner inserted, just add the current node
            smoothed.append(curr)

        smoothed.append(nodes[-1])
        lines[line_name] = smoothed

    # Assign colors to lines
    colors = ["#FF5733", "#33C1FF", "#9D33FF", "#33FF57", "#FFC300"]
    lines_json = []

    for idx, (line_name, nodes) in enumerate(lines.items()):
        lines_json.append({
            'name': line_name,
            'label': line_name,
            'color': colors[idx % len(colors)],
            'shiftCoords': [0, 0],
            'nodes': nodes
        })

    return {
        'stations': stations_meta,
        'lines': lines_json
    }



In [ ]:
def main():
    #csv_path = "/content/drive/MyDrive/KPMG/TubeMap/NCR_Stops_formatted_UTM.csv"  # Update this path if needed
    csv_path = "/content/drive/MyDrive/KPMG/TubeMap/NCR+BRT_v1.csv"
    data = read_csv(csv_path)
    grid_positions = scale_coords(data)
    final_json = build_json(data, grid_positions)

    with open('nairobi_tube_map.json', 'w', encoding='utf-8') as f:
        json.dump(final_json, f, indent=2)

if __name__ == '__main__':
    main()